## Drive Mount + Git Pull Push

1) Run the first cell, when its done click "RESTART RUNTIME"
2) Run the second cell
3) Run the last cell to execute

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/Shareddrives/EECS\ 442\ -\ Project/eecs442-proj
!pip install -r requirements.txt
!pip install pillow --upgrade

Mounted at /content/drive
/content/drive/Shareddrives/EECS 442 - Project/eecs442-proj
     |████████████████████████████████| 11.6MB 10.5MB/s 
     |████████████████████████████████| 2.2MB 54.0MB/s 
     |████████████████████████████████| 81kB 2.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


     |████████████████████████████████| 3.0MB 7.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 8.1.2
    Uninstalling Pillow-8.1.2:
      Successfully uninstalled Pillow-8.1.2


In [1]:
%cd drive/Shareddrives/EECS\ 442\ -\ Project/eecs442-proj
%load_ext autoreload
%autoreload 2

/content/drive/Shareddrives/EECS 442 - Project/eecs442-proj


In [3]:
#!git config --global user.email "wzahm@umich.edu"
#!git config --global user.name "Wyler Zahm"
!git add Executor.ipynb diod
!git commit -m "working gpu training"
!git status
#!git pull
#!git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Executor.ipynb
	deleted:    diode_meta.json
	modified:   metrics.py
	modified:   model_pipeline.py
	modified:   requirements.txt
	modified:   train_model.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	00019_00183_indoors_000_010.png
	__pycache__/
	densenet169-b2777c0a.pth
	diode/val.tar.gz
	models/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")


## Data Downloading

In [ ]:
#!pip install -r requirements.txt
#%cd diode
#!wget http://diode-dataset.s3.amazonaws.com/val.tar.gz
#!tar -zxvf val.tar.gz
#%cd ..

#!wget https://download.pytorch.org/models/densenet169-b2777c0a.pth

Uninstalling matplotlib-3.1.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3-py3.7-nspkg.pth
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/matplotlib/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.7/dist-packages/pylab.py
Proceed (y/n)? y
  Successfully uninstalled matplotlib-3.1.3
  Using cached https://files.pythonhosted.org/packages/ce/63/74c0b6184b6b169b121bb72458818ee60a7d7c436d7b1907bd5874188c55/matplotlib-3.4.1-cp37-cp37m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


## Model Exection

In [2]:
from model_pipeline import main as exec_model
from PIL import Image, ImageFile
import torch

Image.open('./00019_00183_indoors_000_010.png') # Verify PIL is working.
exec_model(use_cuda=True, batch_size=4)

Using GPU.
Batch size: 4
Model sent to GPU


0it [00:00, ?it/s]


Training and Evaluating Basemodel with: 	LR = 0.0001	WD = 0

Epoch #0
107


107it [01:45,  1.02it/s]
107it [00:00, 496.96it/s]
  0%|          | 0/23 [00:00<?, ?it/s]


 RMSE: 4090.8368638653624 MAE: 6.5972758074787174 ARE: inf ALE: nan Thresh 1.25: 0.21650442676009415 Thresh 1.25^2: 0.3704725010001399 Thresh 1.25^3: 0.5064581398651978


...


 RMSE: 3390.0707106545706 MAE: 5.106661930262486 ARE: inf ALE: nan Thresh 1.25: 0.25527441910122783 Thresh 1.25^2: 0.45912440172237023 Thresh 1.25^3: 0.6148740821909682


100%|██████████| 23/23 [00:06<00:00,  3.30it/s]
23it [00:09,  2.54it/s]
0it [00:00, ?it/s]


 RMSE: 88108.69845448369 MAE: 5.586031136305436 ARE: inf ALE: nan Thresh 1.25: 60166.35442142211 Thresh 1.25^2: 114306.36758378622 Thresh 1.25^3: 156191.49677309784

Training MSE	Training Loss	Validation MSE	Validation Loss

3390.0707106545706	tensor(546.4129, device='cuda:0', grad_fn=<AddBackward0>)	88108.69845448369	tensor(115.4670, device='cuda:0')

Epoch #3
107


72it [01:10,  1.01it/s]


KeyboardInterrupt: ignored